In [2]:
!pip install langchain langchain-community faiss-cpu chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 53.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 64.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 64.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 47.0 MB/s eta 0:00:00

In [3]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-large-en-v1.5")

<ipython-input-3-baed1266f386>:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-large-en-v1.5")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  w

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [4]:
import pickle
import pandas as pd
from langchain_community.embeddings import HuggingFaceEmbeddings

# Load cleaned CSV
df = pd.read_csv("cleaned_booking_data.csv")

# Use best embedding model
embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-large-en-v1.5")

# Convert booking data into text
documents = df.astype(str).apply(lambda x: " ".join(x), axis=1).tolist()

# Check if embeddings are already saved
try:
    # Try loading embeddings from file
    with open('embeddings.pkl', 'rb') as f:
        embeddings = pickle.load(f)
    print("Embeddings loaded from file.")
except FileNotFoundError:
    # If embeddings file doesn't exist, generate and save embeddings
    embeddings = embedding_model.embed_documents(documents)

    # Save embeddings to a file
    with open('embeddings.pkl', 'wb') as f:
        pickle.dump(embeddings, f)
    print("Embeddings generated and saved.")


Embeddings generated and saved.


In [6]:
import faiss
import numpy as np

# FAISS Index Creation
dimension = len(embeddings[0])  # Get the dimension of the embeddings (vector size)
index = faiss.IndexFlatL2(dimension)  # Initialize FAISS index (L2 distance)
index.add(np.array(embeddings))  # Add embeddings to the index

# Save FAISS index
faiss.write_index(index, "faiss_index.bin")  # Save the index to disk
print("FAISS index saved to 'faiss_index.bin'.")

FAISS index saved to 'faiss_index.bin'.


In [13]:
from google.colab import drive
import pickle

# Mount Google Drive to Colab
drive.mount('/content/drive')

# Path where you want to save the file in your Google Drive
file_path = '/content/drive/My Drive/embeddings.pkl'

# Let's assume embeddings is the object you want to save
with open(file_path, 'wb') as f:
    pickle.dump(embeddings, f)

print(f"Embeddings saved as 'embeddings.pkl' in Google Drive: {file_path}")


Mounted at /content/drive
Embeddings saved as 'embeddings.pkl' in Google Drive: /content/drive/My Drive/embeddings.pkl


In [8]:
# Load FAISS index
index = faiss.read_index("faiss_index.bin")

def answer_query(query: str, top_k: int = 5):
    """
    Function to process the query and retrieve the most relevant documents from the dataset
    using FAISS and embeddings.
    """
    # Convert the user query into an embedding
    query_embedding = embedding_model.embed_query(query)

    # Perform similarity search
    D, I = index.search(np.array([query_embedding]), top_k)  # D = distances, I = indices

    # Retrieve the most relevant documents
    retrieved_docs = [documents[i] for i in I[0]]

    # Returning the top documents
    return retrieved_docs


# Example Questions:
questions = [
    "Show me total revenue for July 2017.",
    "Which locations had the highest booking cancellations?",
    "What is the average price of a hotel booking?"
]

# Process each question and print the top retrieved documents for each
for question in questions:
    print(f"Query: {question}")
    retrieved_docs = answer_query(question)
    print(f"Top Retrieved Documents: {retrieved_docs}\n")

Query: Show me total revenue for July 2017.
Top Retrieved Documents: ['City Hotel 1 2 2017 July 29 22 0 1 3 0 0 SC PRT Direct Direct 0 0 0 G G 2 No Deposit 0.0 0.0 0 Transient 180.0 0 0 No-Show 2017-07-22 2017-July 3', 'Resort Hotel 1 135 2017 July 28 14 1 2 2 2 0 BB USA Direct Direct 0 0 0 C C 0 No Deposit 250.0 0.0 0 Transient 198.33 0 2 No-Show 2017-07-14 2017-July 12', 'City Hotel 1 4 2017 July 28 9 1 0 2 0 0 BB PRT Direct Direct 0 0 0 A A 0 No Deposit 0.0 0.0 0 Transient 150.0 0 0 No-Show 2017-09-07 2017-July 2', 'City Hotel 0 63 2017 July 28 12 2 5 3 2 0 BB DNK Direct Direct 0 0 0 G G 0 No Deposit 0.0 0.0 0 Transient 300.71 0 2 Check-Out 2017-07-19 2017-July 35', 'Resort Hotel 0 179 2017 July 30 24 2 5 2 2 0 HB DEU Direct Direct 0 0 0 G G 1 No Deposit 250.0 0.0 0 Transient 273.0 0 0 Check-Out 2017-07-31 2017-July 28']

Query: Which locations had the highest booking cancellations?
Top Retrieved Documents: ['City Hotel 1 61 2017 July 29 22 1 1 2 0 0 BB USA Direct Direct 0 0 0 D D 0